
### Exercises: Decision trees, random forests and K-means

In [47]:
# Import necessary libraries here
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

This exercise uses the wine dataset included in the sklearn.datasets module.
In the Jupyter tab explore the 'wine' object. It stores all kinds of data about the wine dataset. You will probably need data, feature_names, target and target_names. In the following cell the wine dataset is loaded.

In [48]:
# Load the wine dataset
wine = load_wine()

## DATA PREPARATION
a. Create an X en y dataset with the features and the target variable. Check the first rows of the datasets.


### Explore data

 a. Investigate the target variable. Which and how many different values are there?\
 b. Do you think these values are nominal, ordinal, interval or ratio? Why?\
 c. Investigate the features (independant variables).\
 d. Are they nominal, ordinal, interval or ratio? Why?

### Prepare data for validation
In order to validate the model, split the dataset into a training and a test set. Use 70% of the data for training and 30% for testing.  \

## Model selection and hyperparameter selection
### 1. Creating a Decision Tree Classifier
a. Based on the type of data in the features and target variable, would you use a classification or regression model?


b. First fit a "DescisionTreeClassifier". Use random_state=42 to make sure that the results are reproducible. In this first run all hyperparameters can be kept on their default values.\
c. Validate the model by using the correct classification metrics

In [56]:
#Validate


Following on this first model you are going to create extra models in the next steps. To make it easy to evaluate each model, create a function evaluate(y_test,y_test_pred) that prints the evalation results like you did in the preveous step.

### 2. Adapting Hyperparameters
a. Create a new model, but now adjust the hyperparameter min_samples_leaf to 11.\
b. What is the effect on the tree?\
c. What is the effect on the metrics?

### 3. Creating and validating a random forest model
With the same dataset we are now going to predict the wine classes using a random forest model. This model uses multiple trees and aggregates the results.\
 a. Create a RandomForerstClassifier with n_estimators hyperparameter set to 100, also use random_state=42\
 b. Fit the model to the training set and evaluate on the test set.\
 c. Based on the metrics, does this model outperform the previous model?\
 d. Lower the hyperparameter n_estimators until it is performing less compared to n_estimators=100? With how many n_estimators the model is performing less.\


### 4. Using another classification model
In the previous exercises you used a Decision Tree and a Random Forest model to predict the wine classes. Now, let's try another classification model.\
a. Go to https://scikit-learn.org/stable/machine_learning_map.html. Start with the start node and go through the nodechart until you find the first model to use. Which model did you find?\
b. Create the model and perform the validation.\
c. What are your findings